In [ ]:
""" WindowData Structure
public class WindowData
{
    // Design Profile
    public int type; // 0: presentaion, 1: participant
    public int userId; // user number
    public int caseNum; // which case 1, 2, 3, 4
    public int windowMode; // 0: path, 1: head
    // Design Details
    public float transparency;
    public float[] position; // center rotation
    public float[] rotation; // eular angle
    public float[] normal; // normal vector
    public List<WindowCorner> corner; // 4 corner rotation: 左上 右上 右下 左下
    public float scale;
}
"""

In [1]:
##### JSON to CSV #####

import json
import pandas as pd
import math
import numpy as np

def getDistance(p):
    if len(p) == 2:
        return math.sqrt(p[0]**2 + p[1]**2)
    elif len(p) == 3:
        return math.sqrt(p[0]**2 + p[1]**2 + p[2]**2)

def cartesian_to_spherical(x, y, z):
    rho = math.sqrt(x**2 + y**2 + z**2)
    theta = math.atan2(y, x) # 經度
    phi = math.acos(z / rho) # 緯度
    theta = -(theta * (180 / math.pi)-90)
    phi = -(phi * (180 / math.pi)-90)
    return theta, phi

def getPhiTheta(corners):
    theta_1, phi_1=cartesian_to_spherical (corners[0]['position'][0], corners[0]['position'][2], corners[0]['position'][1])
    theta_2, phi_2=cartesian_to_spherical (corners[1]['position'][0], corners[1]['position'][2], corners[1]['position'][1])
    theta_3, phi_3=cartesian_to_spherical (corners[2]['position'][0], corners[2]['position'][2], corners[2]['position'][1])
    theta_4, phi_4=cartesian_to_spherical (corners[3]['position'][0], corners[3]['position'][2], corners[3]['position'][1])
    phi_min=min(phi_1, phi_2, phi_3, phi_4)
    phi_max=max(phi_1, phi_2, phi_3, phi_4)
    theta_min=min(theta_1, theta_2, theta_3, theta_4)
    theta_max=max(theta_1, theta_2, theta_3, theta_4)
    return (theta_max-theta_min)#, /4(phi_max-phi_min)

def getVisualAngle(corners):
    topArr = np.array([corners[0]["position"], corners[1]["position"]])
    middleTop = topArr.sum(axis=0) / 2
    botArr = np.array([corners[2]["position"], corners[3]["position"]])
    middleBot = botArr.sum(axis=0) / 2

    # find angle between two vectors
    topUnit = middleTop / np.linalg.norm(middleTop)
    botUnit = middleBot / np.linalg.norm(middleBot)
    angle = np.arccos(np.dot(topUnit, botUnit))
    angle = np.degrees(angle) # vertical

    topArr = np.array([corners[0]["position"], corners[3]["position"]])
    middleTop = topArr.sum(axis=0) / 2
    botArr = np.array([corners[1]["position"], corners[2]["position"]])
    middleBot = botArr.sum(axis=0) / 2

    # find angle between two vectors
    topUnit = middleTop / np.linalg.norm(middleTop)
    botUnit = middleBot / np.linalg.norm(middleBot)
    angle2 = np.arccos(np.dot(topUnit, botUnit))
    angle2 = np.degrees(angle2) #horizontal
    return angle * angle2, angle, angle2


def angle_between_vectors(v, w):
    # 计算向量内积
    dot_product = np.dot(v, w)
    
    # 计算向量的叉积
    cross_product = np.cross(v, w)
    
    # 计算向量的模
    magnitude_v = np.linalg.norm(v)
    magnitude_w = np.linalg.norm(w)
    
    # 计算夹角的余弦值
    cosine_theta = dot_product / (magnitude_v * magnitude_w)
    
    # 计算夹角的弧度
    theta_rad = np.arccos(np.clip(cosine_theta, -1.0, 1.0))
    
    # 根据叉积的方向判断夹角的正负号
    if cross_product > 0:
        theta_rad = -theta_rad
    
    # 将弧度转换为度
    theta_deg = np.degrees(theta_rad)
    
    return theta_deg

def angle_between_vectors_3d(v, w):
    # print(v, w)
    # 右正左負
    # 上正下負
    theta_horizontal_deg = angle_between_vectors(np.array([-v[0], -v[2]]), np.array([-w[0], -w[2]]))
    theta_vertical_deg = angle_between_vectors(np.array([-v[1], -v[2]]), np.array([-w[1], -w[2]]))    
    return theta_horizontal_deg, theta_vertical_deg


def json2csv(inputFile, outputFile):
    data = {
        "type":[],
        "userId":[],
        "caseNum":[],
        "Meeting type":[],
        "environment":[],
        "windowMode":[],
        "center position":[],
        # "rotation":[],
        "distance":[],
        "transparency":[],
        "theta":[],
        "phi":[],
        "pitch":[],
        "row":[],
        "label(81-99)":[],
        "visual angle":[],
        "favorite":[],
        "scale":[],
        "offset": [],
        "x": [], 
        "y": [],
        "z": [],
        "rx": [], 
        "ry": [],
        "rz": [],
        "horlen": [],
        "verlen": []
    }
# 从文件中加载 JSON 数据
    with open(inputFile, 'r') as file:
        json_data = json.load(file)


    # top = []
    # right = []
    # 选择 type=1 且 i=2 的元素
    for id in range(1,17):
        for caseNum in range(1, 5):
            meetingType = 2 if caseNum%2 == 0 else 1
            environment = int((caseNum-1) / 2)
            for window_mode in range(2):
                for type in [1, 0]:
                    # print(id, caseNum, window_mode)
                    selected = [element for element in json_data["windowDatas"] if element["userId"] == id and element["type"] == type and element["caseNum"] == caseNum and element['windowMode'] == window_mode][0]
                    # print(selected)
                    data["type"].append(type)
                    data["userId"].append(id)
                    data["caseNum"].append(caseNum)
                    data["Meeting type"].append(meetingType)
                    data["environment"].append("low" if environment==0 else "high")
                    data["windowMode"].append(window_mode)
                    data["center position"].append('[%.17f, %.17f, %.17f]' % (selected["position"][0], selected["position"][1], selected["position"][2]))
                    # data["rotation"].append('[%.17f, %.17f, %.17f]' % (selected["rotation"][0], selected["rotation"][1], selected["rotation"][2]))
                    # data["rotation"].append([selected["rotation"][0], selected["rotation"][1], selected["rotation"][2]])
                    data["distance"].append(getDistance(selected["position"]))
                    data["transparency"].append(1-selected["transparency"])
                    theta, phi = cartesian_to_spherical(selected["position"][0], selected["position"][2], selected["position"][1])
                    data["theta"].append(theta)
                    data["phi"].append(phi)
                    row, pitch = angle_between_vectors_3d(np.array([selected["position"][0], selected["position"][1], selected["position"][2]]), 
                                                        np.array([selected["normal"][0], selected["normal"][1], selected["normal"][2]]))
                    data["pitch"].append(pitch)
                    data["row"].append(row)
                    data["label(81-99)"].append("")
                    size, verlen, horlen = getVisualAngle(selected["corner"])
                    data["visual angle"].append(size)
                    data["favorite"].append("")
                    data["scale"].append(selected["scale"])
                    # print(selected["scale"])
                    data["offset"].append(getDistance([theta, phi]))
                    data["x"].append(selected["position"][0])
                    data["y"].append(selected["position"][1])
                    data["z"].append(selected["position"][2])
                    data["rx"].append(selected["normal"][0])
                    data["ry"].append(selected["normal"][1])
                    data["rz"].append(selected["normal"][2])
                    data["verlen"].append(verlen)
                    data["horlen"].append(horlen)
    # print(data)
    df = pd.DataFrame(data)
    df.to_csv(outputFile)

    # mean size
    print(df[(df['type']==0)]['visual angle'].mean())
    print(df[(df['type']==1)]['visual angle'].mean())
    print(df[(df['type']==0)]['visual angle'].mean()/df[(df['type']==1)]['visual angle'].mean())


# 打印选择的元素
# print(selected_elements)

In [2]:
json2csv('../AllWindowData.json', "../AllWindowData.csv")
json2csv('../AllWindowData_normalized.json', "../AllWindowData_normalized.csv")
json2csv('../AllWindowData_relative.json', "../AllWindowData_relative.csv")

773.4618896059683
133.22089549708042
5.805860159699339
266.5844755378995
55.28579818166348
4.821934100723845
769.2414315194492
133.49133275367112
5.762482220017348
